In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [1]:

# --- CONFIGURATION ---
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import sys
import json
import re
from unsloth import FastLanguageModel
import torch
from tqdm import tqdm
from IPython.display import display, Markdown

# --- CONFIGURATION ---
INPUT_FILE = "/content/drive/MyDrive/mipd_train_16k.jsonl"
OUTPUT_FILE = "/content/drive/MyDrive/mipd_train_cot_json_strict.jsonl"

TEACHER_MODEL = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"
MAX_SEQ_LENGTH = 4096
MAX_NEW_TOKENS = 512
MAX_RETRIES = 3
SAMPLE_LIMIT = 5 # Set to None for full processing

# --- LOAD MODEL ---
print(f"Loading Teacher Model: {TEACHER_MODEL}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = TEACHER_MODEL,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

# --- RESUME LOGIC ---
processed_inputs = set()
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                entry = json.loads(line)
                # Check raw input to avoid re-processing
                processed_inputs.add(entry['input'])
            except: pass
    print(f"Skipping {len(processed_inputs)} processed samples.")

# --- HELPERS ---
def clean_json_string(text):
    return text.replace("```json", "").replace("```", "").strip()

# --- MAIN LOOP ---
print("Starting Authoritative JSON Generation...")

with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    all_lines = f.readlines()

if SAMPLE_LIMIT:
    all_lines = all_lines[:SAMPLE_LIMIT]

with open(OUTPUT_FILE, 'a', encoding='utf-8') as f_out:

    for i, line in tqdm(enumerate(all_lines), total=len(all_lines), desc="Generating"):
        data = json.loads(line)
        user_input = data['input']

        if user_input in processed_inputs:
            continue

        raw_json_str = clean_json_string(data['output'])

        # --- HARDENED PROMPT ---
        system_prompt = (
            "Jesteś surowym sędzią i ekspertem od dezinformacji. "
            "Twoim zadaniem jest uzasadnienie werdyktu dotyczącego manipulacji w tekście."
        )

        user_prompt = (
            f"TEKST:\n\"{user_input}\"\n\n"
            f"WERDYKT (Techniki): {raw_json_str}\n\n"
            "Napisz uzasadnienie dla tego werdyktu. "
            "Zasady:\n"
            "1. Pisz w trybie orzekającym (np. 'Autor stosuje...', 'Tekst zawiera...'). "
            "2. NIE używaj słów niepewności ('wydaje się', 'być może', 'prawdopodobnie').\n"
            "3. Traktuj podane techniki jako FAKT. Wyjaśnij GDZIE i DLACZEGO występują.\n"
            "4. Jeśli lista technik jest pusta, napisz wprost: 'Tekst ma charakter informacyjny i nie zawiera cech manipulacji.'\n"
            "5. Wygeneruj TYLKO treść uzasadnienia (bez cudzysłowów i formatowania).\n"
        )

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        input_ids = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        success = False
        reasoning_text = ""

        for attempt in range(MAX_RETRIES):
            try:
                with torch.no_grad():
                    outputs = model.generate(
                        input_ids=input_ids,
                        max_new_tokens=MAX_NEW_TOKENS,
                        use_cache=True,
                        temperature=0.7,
                        pad_token_id=tokenizer.eos_token_id
                    )

                generated_text = tokenizer.decode(outputs[0][input_ids.shape[1]:], skip_special_tokens=True).strip()

                # Basic cleanup
                generated_text = generated_text.replace('"', "'") # Replace double quotes to safe single quotes

                if len(generated_text) > 10:
                    reasoning_text = generated_text
                    success = True
                    break
            except:
                continue

        if not success:
            reasoning_text = "Analiza potwierdza występowanie wskazanych technik."

        # --- CONSTRUCT FINAL JSON OBJECT ---
        # We perform the merge here in Python to guarantee valid JSON structure for the UI
        try:
            techniques_list = json.loads(raw_json_str).get("discovered_techniques", [])
        except:
            techniques_list = []

        final_structure = {
            "reasoning": reasoning_text,
            "discovered_techniques": techniques_list
        }

        # Save dataset entry
        new_entry = data.copy()
        # This 'output' is now a perfect stringified JSON ready for SFT
        new_entry['output'] = json.dumps(final_structure, ensure_ascii=False)
        new_entry['original_output'] = raw_json_str

        f_out.write(json.dumps(new_entry, ensure_ascii=False) + "\n")
        f_out.flush()

        if SAMPLE_LIMIT and i < SAMPLE_LIMIT:
            print(f"\n--- Sample {i+1} ---")
            print(json.dumps(final_structure, indent=2, ensure_ascii=False))

print("Done.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading Teacher Model: unsloth/Qwen2.5-7B-Instruct-bnb-4bit...
==((====))==  Unsloth 2026.1.3: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

----------------------------------------

--- Sample 5 ---


**LABELS:** `{"discovered_techniques": []}`

**REASONING:** Tekst ten nie zawiera wyraźnych indywidualnych manipulacji informacyjnych, co odpowiada przypisanej etykiecie brakujących technik manipulacji. Tekst prezentuje informacje naukowe i medyczne o długotrwałych objawach COVID-19 w sposób obiektywny, cytując źródła i podkreślając niepewność naukowców.

Generating: 100%|██████████| 5/5 [01:04<00:00, 12.87s/it]

----------------------------------------
Done! Saved to /content/drive/MyDrive/mipd_train_cot_reasoning_only.jsonl
